# Alpha-Beta-CROWN demos
These three demos are ready to run in Colab with lightweight guidance:
- Image classification certifies a logit ordering inside an L-infinity pixel box.
- Lyapunov checks a controller + Lyapunov candidate on Van der Pol.
- Robot reachability inspects one safe step and one unsafe step side by side.

In [ ]:
import torch

from abcrown import (
    ABCrownSolver,
    ConfigBuilder,
    VerificationSpec,
    input_vars,
    output_vars,
)


## Image classification (expression spec)
- Builds an L-infinity box around a random 28x28 grayscale image.
- Verifies that logit0 is the max of three logits.
- Uses a tiny CNN with random weights.

Step-by-step (modular cells)
- Step 1: Compute graph — tiny conv net mapping 1x28x28 to 3 logits.
- Step 2: Spec — declare symbolic inputs/outputs; constrain inputs to the L-infinity box; enforce logit0 > logit1 and logit0 > logit2.
- Step 3: Config — just use defaults.
- Step 4: Solve — create the solver with spec/model/config, call solve(), and inspect result.status/result.success.



In [ ]:
# Step 1: compute graph (tiny CNN)
class SimpleConvClassifier(torch.nn.Module):
    """CNN that maps 1x28x28 input to 3 logits."""

    def __init__(self) -> None:
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(1, 16, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(16, 16, kernel_size=3, padding=1),
            torch.nn.ReLU(),
        )
        self.head = torch.nn.Linear(16 * 28 * 28, 3)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.ndim == 2:
            x = x.view(x.shape[0], 1, 28, 28)
        feats = self.conv(x)
        flat = feats.view(feats.shape[0], -1)
        return self.head(flat)



In [ ]:
# Step 2: spec builder (symbolic vars + constraints)
def build_image_classification_spec(base_image: torch.Tensor, eps: float):
    """Construct spec with L-infinity box and logit ordering."""
    x = input_vars((1, 28, 28))
    y = output_vars(3)
    input_constraint = (x >= (base_image - eps)) & (x <= (base_image + eps))
    output_constraint = (y[0] > y[1]) & (y[0] > y[2])
    spec = VerificationSpec.build_spec(
        input_vars=x,
        output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )
    return spec



In [ ]:
# Step 3: config (defaults)
def build_demo_config():
    """Config for the image classification spec."""
    return (
        ConfigBuilder.from_defaults()
    )



In [ ]:
# Step 4: solver + one-shot run
def solve_image_classification(base_image: torch.Tensor, eps: float = 0.02):
    """Run ABCrown on the image classification spec."""
    spec = build_image_classification_spec(base_image, eps)
    model = SimpleConvClassifier()
    solver = ABCrownSolver(spec, model, config=build_demo_config())
    result = solver.solve()
    print("[image classification] base eps=", eps)
    print(f"status={result.status}, success={result.success}")
    return result

torch.manual_seed(15)
base_image = torch.rand(1, 1, 28, 28)
_ = solve_image_classification(base_image, eps=0.02)



## Lyapunov (Van der Pol)
- Reuses the tutorial computation graph (controller + Lyapunov NN + dynamics).
- Input box: |x_i| ≤ 0.5.
- Output constraints: V(x) > 0, V_dot < 0.

Step-by-step (modular cells)
- Step 1: Compute graph — assemble dynamics, controller, and Lyapunov networks.
- Step 2: Spec — clamp the state box and require v_min < V(x) < v_max while V_dot(x) < 0.
- Step 3: Config — reuse the defaults with `{"model": {"with_jacobian": True}}`.
- Step 4: Solve — run the solver once and inspect status/success.

In [ ]:
# Step 1: compute graph (Van der Pol + controller + Lyapunov network)
from neural_lyapunov_dependency.computation_graph import (
    Controller,
    Lyapunov,
    VanDerPolDynamics,
    LyapunovComputationGraph,
)

def build_lyapunov_model() -> LyapunovComputationGraph:
    dynamics = VanDerPolDynamics()
    controller = Controller(
        dims=[2, 10, 10, 1],
        x_equilibrium=dynamics.x_equilibrium,
        u_equilibrium=dynamics.u_equilibrium,
        scale=1.0,
    )
    lyapunov = Lyapunov(dims=[2, 40, 40, 1])
    model = LyapunovComputationGraph(dynamics, controller, lyapunov)
    ckpt = "neural_lyapunov_dependency/seed_0.pth"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    try:
        state = torch.load(ckpt, map_location=device)
        state_dict = state["state_dict"] if isinstance(state, dict) and "state_dict" in state else state
        model.load_state_dict(state_dict, strict=False)
    except Exception as e:
        print(f"Failed to load checkpoint from {ckpt}: {e}")
    return model.to(device)


In [ ]:
# Step 2: spec builder (state box + V constraints)
def build_lyapunov_spec():
    x = input_vars(2)
    y = output_vars(2)  # y[0] = V(x), y[1] = V_dot
    # Input box: x0 in [-4.8, 4.8], x1 in [-10.8, 10.8].
    input_constraint = (x >= [-4.8, -10.8]) & (x <= [4.8, 10.8])
    output_constraint = (y[0] < 0.0106) | (y[0] > 0.989) | (y[1] < 0.0)
    return VerificationSpec.build_spec(
        input_vars=x,
        output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )



In [ ]:
# Step 3: config (Lyapunov verification)
def build_lyapunov_config():
    """Config for the Lyapunov verification demo."""
    return (
        ConfigBuilder.from_defaults().set(model__with_jacobian=True)
    )


In [ ]:
# Step 4: solver helper (instantiate solver + print result)
def solve_lyapunov_demo():
    spec = build_lyapunov_spec()
    model = build_lyapunov_model()
    solver = ABCrownSolver(spec, model, config=build_lyapunov_config())
    result = solver.solve()

    print("[lyapunov] verifying Van der Pol controller")

    print(f"status={result.status}, success={result.success}")
    return result

_ = solve_lyapunov_demo()



## Robot reachability (discrete scalar)
- **What is verified:** a one-dimensional closed-loop update $x^+ = g x$ stays inside $[-0.8, 0.8]$ for every $x$ in $[-1, 1]$.
- The dynamics are intentionally tiny so the safe/unsafe outcome is transparent.
- A contraction gain (|g| < 1) keeps the state inside the safety band, while an expansion gain (|g| > 1) pushes some states outside.

Step-by-step (modular cells)
- Step 1: Compute graph — scalar closed-loop model that returns $|x^+|$.
- Step 2: Spec — bound $x$ and enforce a single inequality $0.8 - |x^+| \ge 0$.
- Step 3: Config — build a default config.
- Step 4: Solve — run the safe gain (should certify) and the unsafe gain (should fail).

<img src="figures/drone_demo.png" width="60%">


In [ ]:
# Step 1: compute graph (scalar closed-loop dynamics)
class ScalarClosedLoop(torch.nn.Module):
    """Single-step scalar update that returns |x^+|."""

    def __init__(self, gain: float) -> None:
        super().__init__()
        self.register_buffer("gain", torch.tensor(float(gain), dtype=torch.float32))

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        if state.ndim == 0:
            state = state.unsqueeze(0)
        if state.ndim == 1:
            state = state.unsqueeze(1)
        next_state = self.gain * state
        return next_state.abs()



In [ ]:
# Step 2: spec builder (scalar bounds using expression DSL)
def build_scalar_reachability_spec(initial_bound: float = 1.0, target_bound: float = 0.8) -> VerificationSpec:
    """Reachability spec: |x| <= initial_bound, |x^+| <= target_bound."""
    x = input_vars(1)
    y = output_vars(1)  # y = |x^+|
    input_constraint = (x >= -initial_bound) & (x <= initial_bound)
    output_constraint = y[0] < target_bound
    return VerificationSpec.build_spec(
        input_vars=x,
        output_vars=y,
        input_constraint=input_constraint,
        output_constraint=output_constraint,
    )



In [ ]:
# Step 3: config (discrete scalar reachability)
def build_robot_config():
    """Config for the scalar reachability demo."""
    return (
        ConfigBuilder.from_defaults()
    )



In [ ]:
# Step 4: solver helper (instantiate solver + report stats)
def run_scalar_reachability_case(label: str, gain: float):
    """Instantiate scalar dynamics, solve reachability, and print summary."""

    spec = build_scalar_reachability_spec()
    model = ScalarClosedLoop(gain=gain)
    solver = ABCrownSolver(spec, model, config=build_robot_config())
    result = solver.solve()

    print(f"[scalar {label}] dynamics: x^+ = {gain:.2f} * x")
    print("input bound: |x| <= 1.0, target bound: |x^+| < 0.8")
    print(f"status={result.status}, success={result.success}\n")
    return result


print("Case 1: contraction gain (expected safe)")
_ = run_scalar_reachability_case("safe", gain=0.5)

print("Case 2: expansion gain (expected unsafe)")
_ = run_scalar_reachability_case("unsafe", gain=1.2)
